## James Wilde

#### 6/9/25

#### Homework 5, Part 2

 ### 1) Examine both Open Meteo and Tomorrow and decide which one you'd prefer to use. What drove your decision?

I'm using Open Meteo, because the documentation looks relatively great. The documentation for Tomorrow looks awful.

 ### 2) What is the URL to the documentation? 

https://open-meteo.com/en/docs

### 3) Make a request for the current weather where you are born, or somewhere you've lived.

In [1]:
import requests 
import json 
import os
from dotenv import load_dotenv

url = f"https://api.open-meteo.com/v1/forecast?latitude=40.1001&longitude=-123.795&current=temperature_2m,apparent_temperature,weather_code&wind_speed_unit=mph&temperature_unit=fahrenheit&precipitation_unit=inch"
response = requests.get(url)
data = response.json()

In [2]:
print(f"In Garberville it is {data['current']['temperature_2m']} degrees F!")

In Garberville it is 73.5 degrees F!


### 4) Print out the country this location is in.

This API only includes the latitude and longitude in its output. But we can use Geocoder!

In [3]:
import geocoder

In [4]:
load_dotenv()

api_key = os.getenv("api_key")

In [5]:
g = geocoder.google([data['latitude'], data['longitude']], method='reverse', key=api_key)
country = g.country_long

print(f"This location's country: {country}")

This location's country: United States


### 5) Print out the difference between the current temperature and how warm it feels.

In [6]:
feels_like_difference = round(data['current']['temperature_2m'] - data['current']['apparent_temperature'], 2)

if feels_like_difference > 0:
    print(f"It feels {feels_like_difference} degrees colder than it actually is.")
elif feels_like_difference < 0:
    print(f"It feels {abs(feels_like_difference)} degrees warmer than it actually is.")
else:
    print(f"It feels exactly like the temperature that it is!")

It feels 5.1 degrees warmer than it actually is.


### 6) What's the current temperature at Heathrow International Airport?

In [7]:
g = geocoder.google('Heathrow International Airport', key=api_key)

lat_lhr = g.lat
long_lhr = g.lng

In [8]:
url = f"https://api.open-meteo.com/v1/forecast?latitude={lat_lhr}&longitude={long_lhr}&current=temperature_2m,apparent_temperature,weather_code&wind_speed_unit=mph&temperature_unit=fahrenheit&precipitation_unit=inch"
response = requests.get(url)
lhr_data = response.json()

f"The current temperature at Heathrow International Airport is {lhr_data['current']['temperature_2m']} degrees Fahrenheit."

'The current temperature at Heathrow International Airport is 70.8 degrees Fahrenheit.'

### 7) What URL would I use to request a 3-day forecast at Heathrow?

In [9]:
print(lat_lhr, long_lhr)

51.4679914 -0.455051


For a 3-day forecast of the maximum temperature each day (in Fahrenheit):

https://api.open-meteo.com/v1/forecast?latitude=51.4679914&longitude=-0.455051&daily=temperature_2m_max&forecast_days=3&temperature_unit=fahrenheit

(For more variables, you could add a lot more on to this URL.)

### 8) Print the date of each of the 3 days you're getting a forecast for.

In [10]:
url = f"https://api.open-meteo.com/v1/forecast?latitude=51.4679914&longitude=-0.455051&daily=temperature_2m_max&forecast_days=3&temperature_unit=fahrenheit"
response = requests.get(url)
lhr_3day_data = response.json()

In [11]:
for date in lhr_3day_data['daily']['time']:
    print(date)

2025-06-10
2025-06-11
2025-06-12


### 9) Print the maximum temperature of each of the days.

Hm, the days and temperatures are in separate lists. I want to combine those into a single for loop to print out the date and temperature for each day. From doing some research, it seems like one option is to use the zip() function.

In [12]:
for date, temp in zip(lhr_3day_data['daily']['time'], lhr_3day_data['daily']['temperature_2m_max']):
    print(f"On {date}, the maximum temperature is forecast to be {temp} degrees F.")

On 2025-06-10, the maximum temperature is forecast to be 71.5 degrees F.
On 2025-06-11, the maximum temperature is forecast to be 73.9 degrees F.
On 2025-06-12, the maximum temperature is forecast to be 72.0 degrees F.


### 10) Print only the day with the highest maximum temperature.

To answer this, I took what I learned from question 9 to use zip() to just create a new dictionary with the dates and max temperatures for each date. At first this didn't work, until I learned that I needed to use the dict() function too.

In [13]:
times = lhr_3day_data['daily']['time']
temps = lhr_3day_data['daily']['temperature_2m_max']

paired_temps = dict(zip(times, temps))

print(paired_temps)

{'2025-06-10': 71.5, '2025-06-11': 73.9, '2025-06-12': 72.0}


In [14]:
hottest_day = max(paired_temps, key=paired_temps.get)
print(f"The hottest day is forecasted to be {hottest_day} at {paired_temps[hottest_day]} degrees F.")


The hottest day is forecasted to be 2025-06-11 at 73.9 degrees F.


### 11) Did you find this easier or more difficult than using the weatherapi.com, and why? Which would you recommend to someone interesting in building a tool around weather information?

In general they were actually pretty similar, but I found the weatherapi.com easier simply because it included more location data. Open Meteo seemed to only include latitude and longitude, which made it necessary to use Geocoder to pull in the country, city, etc. More steps = more annoying.